# Clasificación multiclase

Ahora en vez de tener que clasificar entre 2 clases distintas tenemos que clasificar entre más clases. Este problema es casi idéntico al de clasificación binaria, pero a la salida de la red neuronal ya no se puede tener solo una neurona, ya que con una neurona solo podemos tener 1 o 0 (dos clases). Así que veamos cómo se resuelve

Importamos la base de datos de tipos de vinos

In [1]:
from sklearn import datasets

wine = datasets.load_wine()

Podemos ver qué trae esta base de datos

In [2]:
wine.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names'])

La llave `DESCR` es una descripción de la base de datos

In [3]:
print(wine['DESCR'])

.. _wine_dataset:

Wine recognition dataset
------------------------

**Data Set Characteristics:**

    :Number of Instances: 178
    :Number of Attributes: 13 numeric, predictive attributes and the class
    :Attribute Information:
 		- Alcohol
 		- Malic acid
 		- Ash
		- Alcalinity of ash  
 		- Magnesium
		- Total phenols
 		- Flavanoids
 		- Nonflavanoid phenols
 		- Proanthocyanins
		- Color intensity
 		- Hue
 		- OD280/OD315 of diluted wines
 		- Proline

    - class:
            - class_0
            - class_1
            - class_2
		
    :Summary Statistics:
    
    ============================= ==== ===== ======= =====
                                   Min   Max   Mean     SD
    ============================= ==== ===== ======= =====
    Alcohol:                      11.0  14.8    13.0   0.8
    Malic Acid:                   0.74  5.80    2.34  1.12
    Ash:                          1.36  3.23    2.36  0.27
    Alcalinity of Ash:            10.6  30.0    19.5   3.3
    Ma

Además tiene las llaves `data` y `target` donde se encuentran los datos anteriormente descritos. La llave `feature_names` contiene los numbres de cada una de las características

Así que creamos un dataframe con los datos

In [4]:
import pandas as pd

wine_df = pd.DataFrame(wine['data'], columns=wine['feature_names'])
wine_df['target'] = wine['target']
wine_df.head()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0


Vemos las posibles clases que hay

In [5]:
print(wine.target_names)

['class_0' 'class_1' 'class_2']


Vemos cuantos elementos hay de cada clase

In [6]:
wine_df['target'].value_counts()

target
1    71
0    59
2    48
Name: count, dtype: int64

Por último vemos si hay algún dato faltante

In [7]:
wine_df.isnull().sum()

alcohol                         0
malic_acid                      0
ash                             0
alcalinity_of_ash               0
magnesium                       0
total_phenols                   0
flavanoids                      0
nonflavanoid_phenols            0
proanthocyanins                 0
color_intensity                 0
hue                             0
od280/od315_of_diluted_wines    0
proline                         0
target                          0
dtype: int64

## Dataset y Dataloader

Creamos el dataset

In [8]:
import torch

class WineDataset(torch.utils.data.Dataset):
    def __init__(self, dataframe):
        cols = [col for col in dataframe.columns if col != 'target']
        self.parameters = torch.from_numpy(dataframe[cols].values).type(torch.float32)
        self.targets = torch.from_numpy(dataframe['target'].values).type(torch.int64)
        self.targets = torch.nn.functional.one_hot(self.targets, 3).type(torch.float32)

    def __len__(self):
        return len(self.parameters)

    def __getitem__(self, idx):
        parameters = self.parameters[idx]
        target = self.targets[idx]
        return parameters, target

In [9]:
ds = WineDataset(wine_df)
len(ds), len(wine_df)

(178, 178)

Para poder entrenar hemos visto que necesitamos dividir los datos en un conjunto de datos de entrenamiento y en un conjunto de datos de validación. Así que dividimos nuestros datos en estos dos conjuntos.

Como no tenemos muchos datos vamos a dividir el conjunto de datos en un 80% para entrenamiento entrenamiento y un 20% para validación

In [10]:
train_ds, valid_ds = torch.utils.data.random_split(ds, [int(0.8*len(ds)), len(ds) - int(0.8*len(ds))], generator=torch.Generator().manual_seed(42))
len(train_ds), len(valid_ds), len(train_ds) + len(valid_ds)

(142, 36, 178)

Vamos a ver una muestra

In [11]:
sample = train_ds[0]
print(f"len(sample): {len(sample)}")

parameters, target = sample
print(f"parameters: {parameters}\ntype parameters: {type(parameters)}\nparameters.dtype: {parameters.dtype}\nparameters.shape: {parameters.shape}\n\n")
print(f"target: {target}, type target: {type(target)}, target.dtype: {target.dtype}, target.shape: {target.shape}")

len(sample): 2
parameters: tensor([1.2080e+01, 2.0800e+00, 1.7000e+00, 1.7500e+01, 9.7000e+01, 2.2300e+00,
        2.1700e+00, 2.6000e-01, 1.4000e+00, 3.3000e+00, 1.2700e+00, 2.9600e+00,
        7.1000e+02])
type parameters: <class 'torch.Tensor'>
parameters.dtype: torch.float32
parameters.shape: torch.Size([13])


target: tensor([0., 1., 0.]), type target: <class 'torch.Tensor'>, target.dtype: torch.float32, target.shape: torch.Size([3])


Creamos ahora el dataloader

In [12]:
from torch.utils.data import DataLoader

BS_train = 32
BS_val = 1024

train_dl = DataLoader(train_ds, batch_size=BS_train, shuffle=True)
val_dl = DataLoader(valid_ds, batch_size=BS_val, shuffle=False)

Vemos un batch

In [13]:
batch = next(iter(train_dl))
parameters, target = batch[0], batch[1]
type(parameters), parameters.dtype, parameters.shape, type(target), target.shape

(torch.Tensor,
 torch.float32,
 torch.Size([32, 13]),
 torch.Tensor,
 torch.Size([32, 3]))

## Red Neuronal

Creamos una red neuronal para entrenarla

Ahora en la última capa de la red neuronal ponemos una capa con una función de activación, en concreto la función `softmax`.

Como estamos en un problema de clasificación multiclase, a la salida queremos que la red nos de la probabilidad de que la entrada pertenezca a una de las clases, y esto lo conseguimos con la función `softmax`

In [14]:
from torch import nn

class WineNeuralNetwork(nn.Module):
    def __init__(self, num_inputs, num_outputs, hidden_layers=[100, 50, 20]):
        super().__init__()
        self.network = torch.nn.Sequential(
            torch.nn.Linear(num_inputs, hidden_layers[0]),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden_layers[0], hidden_layers[1]),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden_layers[1], hidden_layers[2]),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden_layers[2], num_outputs),
        )
        self.activation = torch.nn.Softmax(dim=1)

    def forward(self, x):
        logits = self.network(x)
        probs = self.activation(logits)
        return logits, probs

Al igual que antes hemos definido una red genérica a la que hay que meterle los tamaños de entrada, de salida, y opcionalmente los tamaños de la capa oculta. Vamos a ver qué tamaño necesitamos a la entrada y a la salida de la red

Un batch tiene unos parámetros con este tamaño

In [15]:
parameters.shape

torch.Size([32, 13])

Tenemos una matriz de tamaño 32x13. 32 es el tamaño del batch size, mientras que 13 es el número de parámetros, por lo que a la entrada necesitamos 13 neuronas

Otra forma de verlo es que como se tiene que hacer una multiplicación matricial de las entradas con la primera capa de la red, si la matriz de entradas tiene un tamaño de 32x13, la matriz que representa las neuronas de la primera capa tiene que tener un tamaño de 31xM. Ya que en una multiplicación matricial, el tamaño de las matrices que se multiplican tienen que ser AxB y BxC, es decir, la dimensión de en medio de las dos matrices tiene que ser la misma

Por otro lado, el mismo batch a la salida tiene un target con este tamaño

In [16]:
target.shape

torch.Size([32, 3])

32 es el tamaño del batch size, pero hay 3 clases, por lo que a la salida queremos que haya 3 neuronas

In [17]:
num_inputs = parameters.shape[1]
num_outputs = target.shape[1]
model = WineNeuralNetwork(num_inputs, num_outputs)

model

WineNeuralNetwork(
  (network): Sequential(
    (0): Linear(in_features=13, out_features=100, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=50, bias=True)
    (3): ReLU()
    (4): Linear(in_features=50, out_features=20, bias=True)
    (5): ReLU()
    (6): Linear(in_features=20, out_features=3, bias=True)
  )
  (activation): Softmax(dim=1)
)

Primero cogemos un batch del dataloader y se lo metemos a la red a ver si funciona y la hemos definido bien

In [18]:
logits, probs = model(parameters)
probs.shape, probs[0:5]

(torch.Size([32, 3]),
 tensor([[9.9718e-01, 2.8174e-03, 1.7664e-06],
         [9.9914e-01, 8.6037e-04, 1.1165e-07],
         [9.9620e-01, 3.7962e-03, 3.4724e-06],
         [9.9023e-01, 9.7409e-03, 2.9274e-05],
         [9.9739e-01, 2.6049e-03, 1.3370e-06]], grad_fn=<SliceBackward0>))

In [19]:
print("De las 32 salidas veamos la primera:")
print(f"La probabilidad de la primera clase ({probs[0][0].item():0.5f}), mas la probabilidad de la segunda clase ({probs[0][1].item():0.5f}), mas la probabilidad de la tercera clase ({probs[0][2].item():0.5f}) suman: {probs[0][0].item() + probs[0][1].item() + probs[0][2].item():0.5f}")

De las 32 salidas veamos la primera:
La probabilidad de la primera clase (0.99718), mas la probabilidad de la segunda clase (0.00282), mas la probabilidad de la tercera clase (0.00000) suman: 1.00000


Si se puede se manda la red a la GPU

In [20]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

model.to(device)

Using cuda device


WineNeuralNetwork(
  (network): Sequential(
    (0): Linear(in_features=13, out_features=100, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=50, bias=True)
    (3): ReLU()
    (4): Linear(in_features=50, out_features=20, bias=True)
    (5): ReLU()
    (6): Linear(in_features=20, out_features=3, bias=True)
  )
  (activation): Softmax(dim=1)
)

Ahora volvemos a probar a meterle un batch

In [21]:
parameters_gpu = parameters.to(device)
logits, probs = model(parameters_gpu)
probs.shape, probs[0:5]

(torch.Size([32, 3]),
 tensor([[9.9718e-01, 2.8174e-03, 1.7664e-06],
         [9.9914e-01, 8.6036e-04, 1.1165e-07],
         [9.9620e-01, 3.7962e-03, 3.4724e-06],
         [9.9023e-01, 9.7409e-03, 2.9274e-05],
         [9.9739e-01, 2.6049e-03, 1.3370e-06]], device='cuda:0',
        grad_fn=<SliceBackward0>))

## Función de pérdida y optimizador

Definimos una función de pérdida y un optimizador

In [22]:
LR = 1e-3

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)


## Ciclo de entrenamiento

Entrenamos la red

In [23]:
num_prints = 4

def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()

    for batch, (X, y) in enumerate(dataloader):
        # X and y to device
        X, y = X.to(device), y.to(device)

        # Compute prediction and loss
        _, probs = model(X)
        loss = loss_fn(probs, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % int(len(dataloader)/num_prints) == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def val_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    model.eval()

    with torch.no_grad():
        for X, y in dataloader:
            # X and y to device
            X, y = X.to(device), y.to(device)
            
            logits, probs = model(X)
            test_loss += loss_fn(probs, y).item()
            correct += (probs.argmax(1) == y.argmax(1)).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

Entrenamos

In [24]:
epochs = 14
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dl, model, loss_fn, optimizer)
    val_loop(val_dl, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 1.164403  [    0/  142]
loss: 1.194580  [   32/  142]
loss: 1.222280  [   64/  142]
loss: 1.231683  [   96/  142]
loss: 1.152400  [   56/  142]
Test Error: 
 Accuracy: 33.3%, Avg loss: 1.139502 

Epoch 2
-------------------------------
loss: 1.115721  [    0/  142]
loss: 1.275453  [   32/  142]
loss: 1.032890  [   64/  142]
loss: 0.995832  [   96/  142]
loss: 1.242216  [   56/  142]
Test Error: 
 Accuracy: 58.3%, Avg loss: 0.992355 

Epoch 3
-------------------------------
loss: 1.017056  [    0/  142]
loss: 1.033365  [   32/  142]
loss: 1.088514  [   64/  142]
loss: 0.953824  [   96/  142]
loss: 1.167778  [   56/  142]
Test Error: 
 Accuracy: 58.3%, Avg loss: 0.996209 

Epoch 4
-------------------------------
loss: 0.999282  [    0/  142]
loss: 1.074593  [   32/  142]
loss: 0.907206  [   64/  142]
loss: 1.116212  [   96/  142]
loss: 0.982325  [   56/  142]
Test Error: 
 Accuracy: 66.7%, Avg loss: 0.963330 

Epoch 5
------------------------

Vamos a meterle un dato del dataset de validación a ver qué tal lo hace la red

In [25]:
valid_parameters, valid_target = next(iter(val_dl))

predictions = model(valid_parameters.to(device))
predictions_logits, predictions_probs = predictions[0], predictions[1]

In [26]:
print(f"para todo el batch de validación")
clases = ["class_0", "class_1", "class_2"]
for i in range(len(valid_target)):
    if predictions_probs[i].argmax().item() == valid_target[i].argmax().item():
        string = "OK"
    else:
        string = "ERROR"
    print(f"\t se esperaba target: {clases[valid_target[i].argmax().item()]}, y se ha predicho: {clases[predictions_probs[i].argmax().item()]} - {string}")

para todo el batch de validación
	 se esperaba target: class_0, y se ha predicho: class_0 - OK
	 se esperaba target: class_1, y se ha predicho: class_0 - ERROR
	 se esperaba target: class_2, y se ha predicho: class_0 - ERROR
	 se esperaba target: class_1, y se ha predicho: class_0 - ERROR
	 se esperaba target: class_0, y se ha predicho: class_0 - OK
	 se esperaba target: class_2, y se ha predicho: class_0 - ERROR
	 se esperaba target: class_1, y se ha predicho: class_0 - ERROR
	 se esperaba target: class_1, y se ha predicho: class_0 - ERROR
	 se esperaba target: class_2, y se ha predicho: class_0 - ERROR
	 se esperaba target: class_0, y se ha predicho: class_0 - OK
	 se esperaba target: class_1, y se ha predicho: class_0 - ERROR
	 se esperaba target: class_0, y se ha predicho: class_0 - OK
	 se esperaba target: class_0, y se ha predicho: class_0 - OK
	 se esperaba target: class_2, y se ha predicho: class_0 - ERROR
	 se esperaba target: class_1, y se ha predicho: class_0 - ERROR
	 se es

Esta red no ha aprendido muy bien